In [ ]:
import os
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Function
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader

from model.dataset import SMPLRandomDataset
from model.SkinningNet import WeightBindingNet
from model.utils import WeightBindingLoss

from meshplot import plot
import igl
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:%s"%device)

In [ ]:
batchSize = 16
trainDataset = SMPLRandomDataset(complexity = "all", 
                                 gender = "mixed", 
                                 dataSize = 5000, 
                                 vertexOrderRandom = True, 
                                 noise = 1, 
                                 rotate = False, 
                                 scale = True,
                                 translate = True,
                                 centre = False)
trainIter = DataLoader(trainDataset, num_workers=0, batch_size=batchSize, shuffle=True, drop_last=True)
allTrainNUM = len(trainDataset)

In [ ]:
net = WeightBindingNet(jointNum = 24)
net = net.to(device)
net.load_state_dict(torch.load(".\stateDict\weightNet_pretrain_rotation.pkl"))

In [ ]:
lr = 0.0001
epochs =100
opt = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = CosineAnnealingLR(opt, epochs, eta_min=0.1*lr)

loss = WeightBindingLoss()

In [ ]:
beginTime = time.time()
net = net.train()

for epoch in range(epochs):
    epochBegin = time.time()
    trainLoss = 0.0
    completedNum = 0
    batchNum = 0
    trainAcc = 0.0
    for V, facesOneRingIdx, rigW, joints in trainIter:
        V = V.to(device).float()
        facesOneRingIdx = facesOneRingIdx.to(device).long()
        rigW = rigW.to(device).float()
        
        opt.zero_grad()
        
        preA = net(V, facesOneRingIdx)
        l, acc = loss(preA, rigW.permute(0, 2, 1))
        l.backward()
        opt.step()
        
        batchLoss = float(l)
        batchAcc = float(acc)
        trainAcc += batchAcc
        trainLoss += batchLoss
        batchNum += 1
        completedNum += batchSize
        compltePer = completedNum/allTrainNUM
        leftTime = ((time.time() - epochBegin)/compltePer - (time.time() - epochBegin))/60
        sys.stdout.write('\r 训练%i / %i, 权重损失: %f, 准确率：%0.2f%%, 当前进度: %0.2f%%, 预计剩余%d分钟'%(completedNum, allTrainNUM, batchLoss, batchAcc*100, compltePer * 100, leftTime))
        sys.stdout.flush()
        
    torch.save(net.state_dict(), ".\stateDict\epochs\weightNet%d.pkl"%epoch)
    print("\n")
    print("epoch%d, 权重损失: %f, 准确率：%0.2f%%, 用时：%0.2f minuetes"%(epoch+1, trainLoss/batchNum, trainAcc/batchNum*100, (time.time() - beginTime) / 60 ))
    print("----------------------------------")
    scheduler.step()

In [ ]:
torch.save(net.state_dict(), ".\stateDict\weightNet_pretrain_noise.pkl")

In [ ]:
batchSize = 8
testDataset = SMPLRandomDataset(complexity = "all", 
                                 gender = "mixed", 
                                 dataSize = 5000, 
                                 vertexOrderRandom = True, 
                                 noise = 0, 
                                 rotate = False, 
                                 scale = False, 
                                 translate = False)
testIter = DataLoader(testDataset, num_workers=0, batch_size=batchSize, shuffle=False, drop_last=False)
allTestNUM = len(testDataset)

In [ ]:
beginTime = time.time()
#net = net.eval()


beginTime = time.time()
testLoss = 0.0
completedNum = 0
batchNum = 0
testAcc = 0.0
for V, facesOneRingIdx, rigW, joints in testIter:
    V = V.to(device).float()
    facesOneRingIdx = facesOneRingIdx.to(device).long()
    rigW = rigW.to(device).float()

    with torch.no_grad():
        preA = net(V, facesOneRingIdx)
        l, acc = loss(preA, rigW.permute(0, 2, 1))


    batchLoss = float(l)
    batchAcc = float(acc)
    testAcc += batchAcc
    testLoss += batchLoss
    batchNum += 1
    completedNum += batchSize
    compltePer = completedNum/allTestNUM
    leftTime = ((time.time() - beginTime)/compltePer - (time.time() - beginTime))/60
    sys.stdout.write('\r 训练%i / %i, 权重损失: %f, 准确率：%0.2f%%, 当前进度: %0.2f%%, 预计剩余%d分钟'%(completedNum, allTestNUM, batchLoss, batchAcc*100, compltePer * 100, leftTime))
    sys.stdout.flush()
print("\n")
print("epoch%d, 权重损失: %f, 准确率：%0.2f%%, 用时：%0.2f minuetes"%(epoch+1, testLoss/batchNum, testAcc/batchNum*100, (time.time() - beginTime) / 60 ))
print("----------------------------------")


In [ ]:
v = V[0].detach().cpu().numpy()

rigw = rigW[0].detach().cpu().numpy()
prew = preA[0].detach().cpu().numpy()

In [ ]:
p = plot(v,c = prew.argmax(axis = 0), shading={"point_size": "0.04"})
#p.add_points(j, shading={"point_size": "0.2", "point_color":"blue"})
#p.add_points(prejoint, shading={"point_size": "0.2", "point_color":"green"})

In [ ]:
testAcc/batchNum